In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd

In [ ]:


# Configurar el navegador
driver = webdriver.Chrome()

# URL base
base_url = 'https://www.alkosto.com/celulares/smartphones/c/BI_101_ALKOS?page={}&sort=relevance'

data = []

# Iterar por las 14 páginas
for page in range(1, 15):  # Páginas de 1 a 14
    url = base_url.format(page)
    print(f"Accediendo a: {url}")
    driver.get(url)

    try:
        # Esperar a que los productos se carguen
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#js-hits li'))
        )
    except TimeoutException:
        print(f"Los productos no se cargaron en la página {page}. Saltando...")
        continue

    # Obtener los elementos de la página actual
    li_elements = driver.find_elements(By.CSS_SELECTOR, '#js-hits li')

    for element in li_elements:
        try:
            # Ajustar los selectores según la estructura del sitio
            titulo_element = element.find_element(By.TAG_NAME, 'h3')
            precio_element = element.find_element(By.CLASS_NAME, 'price')  # Revisa este selector
            marca_element = element.find_element(By.CLASS_NAME, 'product__item__information__brand')

            # Agregar los datos al listado
            data.append({
                'categoria': 'Smartphone',
                'marca': marca_element.text.strip(),
                'nombre_producto': titulo_element.text.strip(),
                'precio': precio_element.text.strip()
            })
        except NoSuchElementException:
            print(f"Elemento faltante en la página {page}. Saltando...")
        except Exception as e:
            print(f"Error desconocido en la página {page}: {e}")

# Cerrar el navegador
driver.quit()

# Convertir los datos a un DataFrame
df = pd.DataFrame(data)

# Mostrar las primeras filas y guardar en un archivo
print(df.head(10))
#df.to_csv('productos_alkosto.csv', index=False)


In [ ]:
df